# Recursion analysis
Beautiful thing - recurrence formulas time analysis is solving recurrence relations! 

Simplest approach - write an exhaustive formula and expand the brackets.

In [1]:
def fact(n):              # T(n)
    if n < 2:             # 1
        return 1          # 0
    return n * fact(n-1)  # 2 + T(n-1) 

$T(1) = 1$

$T(n) = 1 + 2 + T(n - 1) = T(n - 1) + 3$

...

But what if equation involves non-linear terms?

# Master theorem
Master theorem is a beatiful statement, which covers a wide range of algorithms in divide-and-conquer strategy.

In [16]:
# https://stackabuse.com/merge-sort-in-python/

def merge(array, left_index, right_index, middle):
    # Make copies of both arrays we're trying to merge

    # The second parameter is non-inclusive, so we have to increase by 1
    left_copy = array[left_index:middle + 1]
    right_copy = array[middle + 1:right_index + 1]
    
    # Initial values for variables that we use to keep
    # track of where we are in each array
    left_copy_index = 0
    right_copy_index = 0
    sorted_index = left_index

    # Go through both copies until we run out of elements in one
    while left_copy_index < len(left_copy) and right_copy_index < len(right_copy):

        # If our left_copy has the smaller element, put it in the sorted
        # part and then move forward in left_copy (by increasing the pointer)
        if left_copy[left_copy_index] <= right_copy[right_copy_index]:
            array[sorted_index] = left_copy[left_copy_index]
            left_copy_index += 1
        # Opposite from above
        else:
            array[sorted_index] = right_copy[right_copy_index]
            right_copy_index += 1

        # Regardless of where we got our element from
        # move forward in the sorted part
        sorted_index = sorted_index + 1
        
    # We ran out of elements either in left_copy or right_copy
    # so we will go through the remaining elements and add them
    while left_copy_index < len(left_copy):
        array[sorted_index] = left_copy[left_copy_index]
        left_copy_index = left_copy_index + 1
        sorted_index = sorted_index + 1

    while right_copy_index < len(right_copy):
        array[sorted_index] = right_copy[right_copy_index]
        right_copy_index = right_copy_index + 1
        sorted_index = sorted_index + 1

def merge_sort(array, left_index, right_index):
    if left_index > right_index - 1:                   # 2
        return
    middle = (left_index + right_index)//2             # 2
    merge_sort(array, left_index, middle)              # T(n / 2)
    merge_sort(array, middle + 1, right_index)         # T(n / 2)
    merge(array, left_index, right_index, middle)      # O(n)
    
a = [5, 4, 3, 2, 1, 2, 3, 4]
merge_sort(a, 0, len(a)-1)
a

[1, 2, 2, 3, 3, 4, 4, 5]

$\huge T(n) = 2T(\frac{n}{2})+O(n) + 4 = 2T(\frac{n}{2})+O(n)$

## The theorem

If the recurrence relation looks like this:

$\huge{\displaystyle T(n)=a\;T\left({\frac {n}{b}}\right)+f(n),}$

then:

Define a value: $\large{\displaystyle c_{\operatorname {crit} }=\log _{b}a}$.

1. Tail is small. ${\displaystyle f(n)=O(n^{c})}$, and $c < c_{crit}$. Then recursion eats the tail: $T(n)\in\Theta(n^{c_{crit}})=\Theta(n^{log_ba})$.
2. Tail is heavy. ${\displaystyle f(n)=\Omega (n^{c})}$ and $c > c_{crit}$. Then tail eats recurision. $T(n)\in\Theta(f(n))$.
3. Edge case: ${\displaystyle f(n)=\Theta (n^{c_{\operatorname {crit} }}\log ^{k}n)}$. In particular, ${\displaystyle f(n)=\Theta (n^{c_{\operatorname {crit} }})}$ for $k=0$. Then both parts contribute: ${\displaystyle T(n)=\Theta \left(n^{c_{\operatorname {crit} }}\log ^{k+1}n\right)}$.

## Tail is small

### Karatsuba multiplication

In [17]:
#### How we multiply on paper

def mult(first, second, base=10):
  result = 0
  for c in second:
    subresult = 0
    for d in first:
      subresult *= base
      subresult += int(d, base) * int(c, base)
    result *= base
    result += subresult
  return result

mult('123', '999')

122877

In [ ]:
def cmp(a, b):
    if len(a) > len(b): return 1
    if len(a) < len(b): return -1
    for c1, c2 in zip(a, b):
        if c1 > c2: return 1
        elif c1 < c2: return -1
    return 0

def isnegative(a):
    return a[0] == "-"

def flip(a):
    if a == "0": return a
    elif a[0] == "-": return a[1:]
    else: return "-" + a

def minus(a, b):
    # replace minus with plus if possible
    if isnegative(a) ^ isnegative(b):
        if isnegative(a):
            return flip(plus(flip(a), b))
        else:
            return plus(a, flip(b))
        
    if isnegative(a) and isnegative(b):
        return flip(minus(flip(a), flip(b)))
        
    # only positive here
    c = cmp(a, b)
    if c == 0:
        return "0"
    if c < 0:
        print("!", a, b)
        return flip(minus(b, a))
    
    result = ""
    bonus = 0
    la = len(a)
    lb = len(b)

    for i in range(max(la, lb)):
        ca = int(a[la - i - 1]) if la - i > 0 else 0
        cb = int(b[lb - i - 1]) if lb - i > 0 else 0
        s = ca - cb + bonus
        print(ca, cb, s, bonus, result)
        result = str(s % 10) + result
        bonus = s // 10
    return result

def plus(a, b):
    # replace plus with minus if possible
    if isnegative(a) ^ isnegative(b):
        if isnegative(a):
            return minus(b, a)
        else:
            return minus(a, b)
    if isnegative(a) and isnegative(b):
        return flip(plus(a, b))
    
    la = len(a)
    lb = len(b)
    result = ""
    bonus = 0
    for i in range(max(la, lb)):
        ca = int(a[la - i - 1]) if la - i > 0 else 0
        cb = int(b[lb - i - 1]) if lb - i > 0 else 0
        s = ca + cb + bonus
        result = str(s % 10) + result
        bonus = s // 10
    if bonus:
        result = str(bonus) + result
    return result

def karatsuba():
    
